**Nota:** Aquest quadern està dissenyat per a **Google Colab**.

Si veieu el logotip de Colab <span style='vertical-align:bottom;'><img src='https://colab.research.google.com/img/colab_favicon_256px.png' width='40' alt='Colab logo'></span> a la cantonada superior esquerra, ja ho teniu tot a punt! Si us plau, **continueu**.

Si no veieu el logotip (per exemple, si esteu a GitHub), feu clic al botó de sota per obrir-lo a l'entorn correcte:

[![Obrir a Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mparrott-at-wiris/aimodelshare/blob/master/notebooks/justice_and_equity_advance_notebook_ca.ipynb)

# **Repte Avançat de Justícia i Equitat: Construir i Enviar Models Personalitzats**

Benvinguts a la **Via Avançada** del Repte de Justícia d'Ètica en Joc (Ethics at Play).

**A qui va dirigit?**
Aquest quadern està dissenyat per a participants amb experiència en Python (per exemple, Scikit-Learn, TensorFlow, PyTorch). En lloc d'utilitzar les aplicacions ludificades, construireu, entrenareu i enviareu els vostres propis models d'aprenentatge automàtic directament a la classificació de la competició.

**L'Objectiu:**
Entrenar un model per predir el risc de reincidència (la probabilitat de tornar a delinquir) utilitzant el conjunt de dades COMPAS, equilibrant la precisió i l'equitat.

## 🚀 **Guia d'Inici Ràpid**

Per participar en el repte, completeu aquests 5 passos:

1.  **Instal·lar Llibreries:** Executeu la cel·la de configuració per instal·lar `aimodelshare`.
2.  **Obtenir les Dades:** Executeu la cel·la de càrrega de dades per recuperar les dades d'entrenament i prova pre-dividides.
3.  **Entrenar el Vostre Model:** Utilitzeu l'exemple de Pipeline de Scikit-Learn proporcionat o escriviu el vostre propi codi d'entrenament personalitzat.
4.  **Connectar:** Enllaceu aquest quadern a la Classificació del Repte de Justícia.
5.  **Enviar:** Envieu les vostres prediccions a la classificació per veure la vostra puntuació.

**Preparats? Feu clic al botó de reproducció ▶ a la primera cel·la de sota per començar.**

---
# **Pas 1: Instal·lació**

Hem d'instal·lar la llibreria `aimodelshare` per connectar-nos al backend de la competició.

In [ ]:
# Instal·lar la llibreria aimodelshare
print("Instal·lant les llibreries necessàries...")
!pip install aimodelshare --upgrade -q --no-warn-script-location > /dev/null 2>&1
print("✅ Instal·lació completada!")

---
# **Pas 2: Carregar Dades**

Carregarem les dades d'entrenament i prova directament des de les URL oficials de la competició.

* **X_train:** Característiques per a l'entrenament.
* **y_train:** Etiquetes objectiu (s'ha produït reincidència?) per a l'entrenament.
* **X_test:** Característiques per a la prova (generareu prediccions sobre això).

In [ ]:
import pandas as pd

# 1. Carregar Dades des de les URL
X_train = pd.read_csv("https://raw.githubusercontent.com/AIModelShare/aimodelshare_tutorials/refs/heads/main/datasets/ethicsatplay/X_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/AIModelShare/aimodelshare_tutorials/refs/heads/main/datasets/ethicsatplay/X_test.csv")
y_train_labels = pd.read_csv("https://raw.githubusercontent.com/AIModelShare/aimodelshare_tutorials/refs/heads/main/datasets/ethicsatplay/y_train.csv")

# Assegurar que y_train és una sèrie 1D (necessari per a sklearn)
y_train = y_train_labels.squeeze()

# 2. Definir Llistes de Característiques per al Pipeline
# Aquestes llistes coincideixen amb les columnes presents a X_train i X_test
ALL_NUMERIC_COLS = ["juv_fel_count", "juv_misd_count", "juv_other_count", "days_b_screening_arrest", "age", "length_of_stay", "priors_count"]
ALL_CATEGORICAL_COLS = ["race", "sex", "c_charge_degree", "c_charge_desc"]

print("✅ Dades carregades correctament!")
print(f"Forma de les dades d'entrenament: {X_train.shape}")
print(f"Forma de les dades de prova: {X_test.shape}")
print("\nPrimeres 5 files de les dades d'entrenament:")
X_train.head()

---
# **Pas 3: Entrenar el Model amb la Llibreria Preferida (Sklearn, Tensorflow, Pytorch, etc.)**

Utilitzarem un **Pipeline de Scikit-Learn** per agilitzar el preprocessament i el modelatge.

Aquest pipeline:
1.  **Imputar Valors Perduts** (Omplir els NaNs amb la mitjana per als números, el més freqüent per a les categories).
2.  **Codificar One-Hot** les columnes categòriques (Raça, Sexe, Grau del Càrrec, Descripció del Càrrec).
3.  **Escalar** les columnes numèriques (Edat, Antecedents, Durada de l'Estada, etc.).
4.  **Entrenar** un classificador de Regressió Logística.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# 1. Definir grups de característiques utilitzant constants del Pas 2
# Les característiques numèriques s'imputaran i escalaran
numeric_features = ALL_NUMERIC_COLS

# Les característiques categòriques s'imputaran i es codificaran amb One-Hot
categorical_features = ALL_CATEGORICAL_COLS

# 2. Definir Transformadors amb Imputació
# Numèric: Imputar valors perduts amb la mitjana, després escalar
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categòric: Imputar valors perduts amb el valor més freqüent, després codificar amb One-Hot
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Crear Preprocessador utilitzant els transformadors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4. Crear Pipeline (Preprocessador + Model)
# Podeu substituir LogisticRegression per qualsevol altre model de sklearn (per exemple, RandomForestClassifier)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# 5. Entrenar el pipeline
pipeline.fit(X_train, y_train)

# 6. Generar prediccions al conjunt de prova
predictions = pipeline.predict(X_test)

print(f"✅ Model Entrenat! Prediccions generades per a {len(predictions)} mostres.")
print("Procediu al següent pas per enviar les vostres prediccions a la classificació.")

---
# **Pas 4: Connectar a la Classificació**

Aquest pas connecta el vostre quadern al backend específic per al Repte de Justícia i Equitat.

*Nota: Se us demanarà que introduïu un nom d'usuari i una contrasenya. Si no en teniu, haureu de crear-ne un a [modelshare.ai](https://www.modelshare.ai)

In [ ]:
from aimodelshare.aws import set_credentials
from aimodelshare.playground import Competition
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# L'URL específic del Model Playground per al Repte de Justícia
my_playground_url = "https://cf3wdpkg0d.execute-api.us-east-1.amazonaws.com/prod/m"

# Establiu les vostres credencials (apareixerà una finestra emergent)
set_credentials(apiurl=my_playground_url)

# Genereu el vostre testimoni d'accés a la sessió
token=os.getenv("AWS_TOKEN")

# Connectar a la competició
playground = Competition(my_playground_url)

---
# **Pas 5: Enviar i Comprovar Resultats**

Envieu les vostres prediccions a la classificació.

In [ ]:
# 1. Envieu les vostres prediccions
# Nota: Passem None per al model i el preprocessador perquè només enviem prediccions per a l'avaluació
playground.submit_model(
    model=None,
    preprocessor=None,
    prediction_submission=predictions, token=token,
    input_dict={
        "Team": "The Ethical Explorers", # Canvieu el nom de l'equip manualment si cal.
        "description": "Logistic Regression with Sklearn Pipeline",
        "tags": "sklearn, logistic_regression, advanced_pathway, pipeline"
    }
)

print("✅ Prediccions enviades correctament!")

# 2. Comprovar la classificació
print("Carregant classificació...")
leaderboard = playground.get_leaderboard()
playground.stylize_leaderboard(leaderboard)